In [1]:
import tensorflow as tf 
from tensorflow import keras 

import numpy as np 
import matplotlib.pyplot as plt

In [2]:
# 파일 불러오기
import os, re, glob
import cv2

image_size = 32

groups_folder_path = './IAAFT/LineGraph/'

X = []
y = []

for label in ['Bi', 'Tri']:    
    
    for top, dir, f in os.walk(groups_folder_path+label+"/"):
        for filename in f:
            img = cv2.imread(groups_folder_path+label+"/"+filename)
            img = cv2.resize(img, None, fx=image_size/img.shape[0], fy=image_size/img.shape[1])
            X.append(img/256) 

            if label == 'Bi':
                y.append(2)
            else:
                y.append(3)
                
X = np.array(X)
y = np.array(y)

In [3]:
# train, test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [4]:
print(X_train.shape)

(16000, 21, 48, 3)


In [5]:
import collections

collections.Counter(y_train)

Counter({3: 8000, 2: 8000})

In [6]:
# validation split

X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.1, random_state=10, stratify=y_train)

In [7]:
print(X_train.shape, X_validation.shape)
print(collections.Counter(y_train))

(14400, 21, 48, 3) (1600, 21, 48, 3)
Counter({2: 7200, 3: 7200})


In [8]:
AlexNet = keras.models.Sequential([
        keras.layers.Conv2D(256, (6, 6), activation='relu', input_shape=X_train[0].shape), 
        keras.layers.MaxPooling2D( (2, 2) ), 
        keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'), 
        keras.layers.Conv2D(384, (3, 3), padding='same', activation='relu'), 
        keras.layers.Conv2D(256, (3, 3), padding='same', activation='relu'), 
        keras.layers.MaxPooling2D( (2, 2) ), 
        keras.layers.Flatten(), 
        keras.layers.Dense(4096, activation='relu'), 
        keras.layers.Dense(4096, activation='relu'), 
        keras.layers.Dense(len(y_train), activation='softmax') 
])

In [9]:
AlexNet.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 16, 43, 256)       27904     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 8, 21, 256)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 21, 384)        885120    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 21, 384)        1327488   
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 21, 256)        884992    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 10, 256)        0         
_________________________________________________________________
flatten (Flatten)            (None, 10240)             0

In [10]:
AlexNet.compile(loss=keras.losses.sparse_categorical_crossentropy, 
              optimizer=keras.optimizers.SGD(learning_rate=0.01), 
              metrics=[keras.metrics.sparse_categorical_accuracy])

In [11]:
AlexNet_history = AlexNet.fit(X_train, y_train, epochs=30, 
                    validation_data=(X_validation, y_validation))

Train on 14400 samples, validate on 1600 samples
Epoch 1/30
14400/14400 [==============================] - 344s 24ms/sample - loss: 1.4403 - sparse_categorical_accuracy: 0.4945 - val_loss: 0.7136 - val_sparse_categorical_accuracy: 0.5000
Epoch 2/30
14400/14400 [==============================] - 348s 24ms/sample - loss: 0.7141 - sparse_categorical_accuracy: 0.4994 - val_loss: 0.6973 - val_sparse_categorical_accuracy: 0.5000
Epoch 3/30
14400/14400 [==============================] - 347s 24ms/sample - loss: 0.7115 - sparse_categorical_accuracy: 0.5016 - val_loss: 0.6953 - val_sparse_categorical_accuracy: 0.5000
Epoch 4/30
14400/14400 [==============================] - 347s 24ms/sample - loss: 0.7080 - sparse_categorical_accuracy: 0.5083 - val_loss: 0.6947 - val_sparse_categorical_accuracy: 0.5000
Epoch 5/30
14400/14400 [==============================] - 347s 24ms/sample - loss: 0.7039 - sparse_categorical_accuracy: 0.5135 - val_loss: 0.6887 - val_sparse_categorical_accuracy: 0.5031
Epoch 

In [12]:
AlexNet_history = AlexNet.fit(X_train, y_train, epochs=5, 
                    validation_data=(X_test, y_test))

Train on 14400 samples, validate on 4000 samples
Epoch 1/5
14400/14400 [==============================] - 356s 25ms/sample - loss: 0.0070 - sparse_categorical_accuracy: 0.9978 - val_loss: 0.0077 - val_sparse_categorical_accuracy: 0.9983
Epoch 2/5
14400/14400 [==============================] - 357s 25ms/sample - loss: 0.0084 - sparse_categorical_accuracy: 0.9970 - val_loss: 0.0095 - val_sparse_categorical_accuracy: 0.9965
Epoch 3/5
14400/14400 [==============================] - 356s 25ms/sample - loss: 0.0050 - sparse_categorical_accuracy: 0.9981 - val_loss: 0.0064 - val_sparse_categorical_accuracy: 0.9983
Epoch 4/5
14400/14400 [==============================] - 356s 25ms/sample - loss: 0.0059 - sparse_categorical_accuracy: 0.9981 - val_loss: 0.0059 - val_sparse_categorical_accuracy: 0.9985
Epoch 5/5
14400/14400 [==============================] - 356s 25ms/sample - loss: 0.0058 - sparse_categorical_accuracy: 0.9981 - val_loss: 0.0101 - val_sparse_categorical_accuracy: 0.9973
